In [2]:
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import csv


In [3]:
NAMESPACES = {
    'EurLex': {'akn': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0'},
    'Normattiva': {'akn': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0'},
    'PDL': {'akn': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0/WD17'}
}

### Count definitions

In [4]:
def checkXML(xmlfile, dataset):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    namespace = NAMESPACES[dataset]

    definitions_el = root.findall('.//akn:definitions', namespace)
    if definitions_el:
        return True
    else:
        return False


datasets = ['EurLex', 'Normattiva', 'PDL']

for dataset in datasets:
    extracted, er, c = 0, 0, 0
    target = Path('../data/datasets/' + dataset)

    for file in target.rglob('*.xml'):
        c += 1
        try:
            definitions = checkXML(file, dataset)
            if definitions:
                extracted += 1
        except Exception as e:
            #print(e, file)
            er += 1
            continue

    print(f'{dataset}: found definitions in {extracted} out of {c} files. {er} errors')

EurLex: found definitions in 889 out of 15283 files. 0 errors
Normattiva: found definitions in 401 out of 3195 files. 0 errors
PDL: found definitions in 78 out of 3709 files. 0 errors


### Extract definitions

In [105]:
def parseXML(xmlfile, dataset):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    namespace = NAMESPACES[dataset]

    definitions_el = root.findall('.//akn:definitions', namespace)
    if not definitions_el:
        return None

    definitions = []
    for definition in root.findall('.//akn:definition', namespace):
        definendum, definiens, references, full_def = parse_definition(definition, root, namespace)
        if definendum:
            definitions.append({
                'def_n': definition.find('.//akn:definitionHead', namespace).attrib.get('href', ''),
                'label': definition.find('.//akn:definitionHead', namespace).attrib.get('refersTo', ''),
                'definendum': clean_definendum(definendum),
                'definiens': clean_definiens(definiens),
                'full_definition': clean_full_def(full_def),
                'references': references,
                'provenance': dataset,
                'document': xmlfile.name
            })
    return definitions


def parse_definition(definition, root, namespace):
    """Extracts definendum, definiens, and references from a single definition."""
    definition_head = definition.find('.//akn:definitionHead', namespace)
    definition_body_elements = definition.findall('.//akn:definitionBody', namespace)

    definendum_id = definition_head.attrib.get('href', '').lstrip('#')
    try:
        definendum = root.find(f".//akn:def[@eId='{definendum_id}']", namespace).text
    except AttributeError as e:
        definendum = None
    
    try:
        full_def = "".join(root.find(f".//*[@defines='#{definendum_id}']").itertext())
    except Exception as e:
        full_def = None

    definiens = []
    references = []
    for body in definition_body_elements:
        body_text, body_references = extract_body_and_references(body, root, namespace)
        if body_text:
            definiens.append(body_text)
            references.extend(body_references)

    return definendum, ' '.join(definiens), references, full_def


def extract_body_and_references(body, root, namespace):
    """Extracts text and references from a single definition body."""
    body_id = body.attrib.get('href', '').lstrip('#')
    body_element = root.find(f".//akn:defBody[@eId='{body_id}']", namespace)
    if body_element is not None:
        body_text = ''.join(body_element.itertext())
        references = [
            ref.attrib.get('href', '') for ref in body_element.findall('.//akn:ref', namespace)
        ]
        return body_text, references
    return None, None


def clean_definendum(text: str):
    text = text.strip()
    if text.startswith(("«", "\'", "\"")) and text.endswith(("»", "\'", "\"")):
        text = text[1:-1]

    return text.strip()


def clean_definiens(text: str):
    if text.startswith(':'):
        text = text.lstrip(':')
    if text.startswith(','):
        text = text.lstrip(',')

    text = text.strip().replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)

    return text


def clean_full_def(text: str):
    text = text.strip().replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('""', '')
    
    return text

def save_definitions(definitions, output_file):
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    Path(output_file).touch()
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=definitions[0].keys(), delimiter="\t", quoting=csv.QUOTE_MINIMAL)
        writer.writeheader()
        for row in definitions:
            writer.writerow(row)

In [ ]:
# single file test


def parse_definition(definition, root, namespace):
    """Extracts definendum, definiens, and references from a single definition."""
    definition_head = definition.find('.//akn:definitionHead', namespace)
    definition_body_elements = definition.findall('.//akn:definitionBody', namespace)

    definendum_id = definition_head.attrib.get('href', '').lstrip('#')
    definendum = root.find(f".//akn:def[@eId='{definendum_id}']", namespace).text
    
    try:
        full_def = "".join(root.find(f".//*[@defines='#{definendum_id}']").itertext())
    except Exception as e:
        full_def = None

    definiens = []
    references = []
    for body in definition_body_elements:
        xmlstr = ET.tostring(body, encoding='unicode')
        print(xmlstr)
        body_text, body_references = extract_body_and_references(body, root, namespace)
        if body_text:
            definiens.append(body_text)
            references.extend(body_references)

    return definendum, ' '.join(definiens), references, full_def


def extract_body_and_references(body, root, namespace):
    """Extracts text and references from a single definition body."""
    body_id = body.attrib.get('href', '').lstrip('#')
    body_element = root.find(f".//akn:defBody[@eId='{body_id}']", namespace)
    if body_element is not None:
        body_text = ''.join(body_element.itertext())
        references = [
            ref.attrib.get('href', '') for ref in body_element.findall('.//akn:ref', namespace)
        ]
        return body_text, references
    return None, None

def parseXML(xmlfile, dataset):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    namespace = NAMESPACES[dataset]

    definitions_el = root.findall('.//akn:definitions', namespace)
    if not definitions_el:
        return None

    definitions = []
    for definition in root.findall('.//akn:definition', namespace):
        definendum, definiens, references, full_def = parse_definition(definition, root, namespace)
        definitions.append({
            'def_n': definition.find('.//akn:definitionHead', namespace).attrib.get('href', ''),
            'label': definition.find('.//akn:definitionHead', namespace).attrib.get('refersTo', ''),
            'definendum': clean_definendum(definendum),
            'definiens': clean_definiens(definiens),
            'full_definition': clean_full_def(full_def),
            'references': references,
            'provenance': dataset,
            'document': xmlfile.name
        })
    return definitions


t = '/home/leo/Desktop/dhdk/Master thesis/.project/data/datasets/Normattiva/20210227_21G00022_VIGENZA_20240101.xml'
t2 = '/home/leo/Desktop/dhdk/Master thesis/.project/data/datasets/EurLex/32014L0049.xml'
t3 = '/home/leo/Desktop/dhdk/Master thesis/.project/data/datasets/EurLex/32021R0403.xml'
t4 = '/home/leo/Desktop/dhdk/Master thesis/.project/data/datasets/EurLex/32010L0035.xml'
t5 = '/home/leo/Desktop/dhdk/Master thesis/.project/data/datasets/EurLex/32013R1293.xml'

definitions = parseXML(Path(t5), 'EurLex')

#save_definitions(definitions, '/home/leo/Desktop/dhdk/Master thesis/.project/data/definitions/definitions.tsv')
definitions

<div class="alert">
  <p>Do we need to format the lists (e.g. with (a), (b), etc.)?? Can it be beneficial to RAG?</p>
</div> 


Ciao Michele, due dubbi:
1. Forse ne avevamo già parlato ma quello non c'è un dataset migliore per EurLex? Perché ci sono errori di annotazione abbastanza importanti (tipo blocchi di definizioni con attributo Source "unibio" che vanno ad interferire con altri blocchi con Source "unibo", questo in 72 casi, oppure definizioni che a causa della presenza di virgolette singole e doppie (e.g. ‘status free from “disease”  ’ means a disease-free status of...) sono uscite annotate male e risultano come stringhe vuote).
2. Conviene fare gli embedding di definendum e definiens ("x means y") o di separarli (o quantomeno di fare l'embedding dei definendi e della stringa completa in due db diversi)?

In [54]:
datasets = ['EurLex', 'Normattiva', 'PDL']

extracted, er, c = 0, 0, 0
for dataset in datasets:
    target = Path('../data/datasets/' + dataset)

    for file in target.rglob('*.xml'):
        c += 1
        try:
            definitions = parseXML(file, dataset)
            if definitions:
                extracted += 1
                save_definitions(definitions, Path('../data/definitions2') / file.name.replace('.xml', '.tsv'))
        except Exception as e:
            print(e, file)
            er += 1
            continue

print(f'extracted definitions from {extracted} files. {er} errors out of {c} files.')

'NoneType' object has no attribute 'strip' ../data/datasets/EurLex/32013R0608.xml
'NoneType' object has no attribute 'strip' ../data/datasets/EurLex/32011R1227.xml
'NoneType' object has no attribute 'strip' ../data/datasets/EurLex/32010R0206R(01).xml
extracted definitions from 1365 files. 3 errors out of 22187 files.


72 EurLex documents have a typo in the declaration of the definitions. There are two distinct <definitions> tags that, one of which is sourced to "unibio" and the second to "unibo". Inside of each element, the ids of the definitions start in both cases from 1, resulting in duplicate references.

In [77]:
import polars as pl

df = pl.read_parquet('../data/definitions/definitions.parquet')#, has_header=False, schema={'label': pl.Utf8, 'showAs': pl.Utf8, 'definition_body': pl.Utf8}, truncate_ragged_lines=True)
df

id,definition_text,def_n,dataset,document_id,references,frbr_work,frbr_expression
u32,str,str,str,str,list[str],str,str
0,"""obstacles to trade: means any …","""#def_1""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
1,"""injury: means any material inj…","""#def_2""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
2,"""adverse trade effects: means t…","""#def_3""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
3,"""Union industry: means either: …","""#def_4""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
4,"""Union enterprise: means a comp…","""#def_5""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
…,…,…,…,…,…,…,…
14724,"""contratto di sponsorizzazione:…","""#def_8""","""PDL""","""18PDL0040890_PD.xml""",[],"""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…"
14725,"""rappresentanti di interessi: i…","""#def_1""","""PDL""","""19PDL0004650_PD.xml""","[""#item_b""]","""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…"
14726,"""decisori pubblici: i membri de…","""#def_2""","""PDL""","""19PDL0004650_PD.xml""",[],"""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…"


In [110]:
df = pl.read_csv('../data/definitions2/*.tsv', separator='\t')#, has_header=False, schema={'label': pl.Utf8, 'showAs': pl.Utf8, 'definition_body': pl.Utf8}, truncate_ragged_lines=True)
df

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str
"""#def_1""","""#georeferenziazione""","""georeferenziazione""","""tecnica di attribuzione di coordinate geografiche a un oggetto grafico, usata nelle procedure di cartografia computerizzata e nella costruzione di basi cartografiche digitali;""","""a) georeferenziazione : tecnica di attribuzione di coordinate geografiche a un oggetto grafico, usata nelle procedure di cartografia computerizzata e nella costruzione di basi cartografiche digitali;""","""[]""","""PDL""","""18PDL0001470_PD.xml"""
"""#def_2""","""#sistemaInformativoGeografico""","""sistema informativo geografico""","""sistema informatico, hardware e software, utilizzato per la memorizzazione e per il processamento di dati geografici nelle modalità grafica e analitica.""","""b) sistema informativo geografico : sistema informatico, hardware e software, utilizzato per la memorizzazione e per il processamento di dati geografici nelle modalità grafica e analitica.""","""[]""","""PDL""","""18PDL0001470_PD.xml"""
"""#def_1""","""#gestoreDelSottoprodottoUmido(Gsu)""","""gestore del sottoprodotto umido (GSU)""","""il soggetto pubblico o privato che gestisce i sottoprodotti organici ai fini e con le modalità stabiliti dalla presente legge;""","""a) gestore del sottoprodotto umido (GSU) : il soggetto pubblico o privato che gestisce i sottoprodotti organici ai fini e con le modalità stabiliti dalla presente legge;""","""[]""","""PDL""","""18PDL0001550_PD.xml"""
"""#def_2""","""#scartiAlimentari""","""scarti alimentari""","""sottoprodotti della produzione, della lavorazione e della preparazione di prodotti alimentari;""","""b) scarti alimentari : sottoprodotti della produzione, della lavorazione e della preparazione di prodotti alimentari;""","""[]""","""PDL""","""18PDL0001550_PD.xml"""
"""#def_3""","""#compostaggio""","""compostaggio""","""fermentazione della materia organica ottenuta con l'utilizzo di ossigeno e dei microrganismi naturalmente presenti nella stessa o dei microrganismi naturalmente presenti o selezionati durante lo svolgimento del processo stesso;""","""c) compostaggio : fermentazione della materia organica ottenuta con l'utilizzo di ossigeno e dei microrganismi naturalmente presenti nella stessa o dei microrganismi naturalmente presenti o selezionati durante lo svolgimento del processo stesso;""","""[]""","""PDL""","""18PDL0001550_PD.xml"""
"""#def_4""","""#luoghiDiRistorazione""","""luoghi di ristorazione""","""le mense pubbliche, convenzionate o private ovvero che svolgono in qualsiasi modo servizio pubblico di ristorazione collettiva, comprese le mense aziendali e interaziendali, le mense che svolgono servizi di ristorazione prescolastica, scolastica o universitaria, ospedaliera, militare o penitenziaria…","""d) luoghi di ristorazione : le mense pubbliche, convenzionate o private ovvero che svolgono in qualsiasi modo servizio pubblico di ristorazione collettiva, comprese le mense aziendali e interaziendali, le mense che svolgono servizi di ristorazione prescolastica, scolastica o universitaria, ospedalie…","""[]""","""PDL""","""18PDL0001550_PD.xml"""
"""#def_5""","""#rifiutoOrganico""","""rifiuto organico""","""i rifiuti biodegradabili di giardini e parchi, i rifiuti alimentari e di cucina prodotti da nuclei domestici, ristoranti, servizi di ristorazione e punti di vendita al dettaglio e i rifiuti simili prodotti dall'industria alimentare raccolti in modo differenziato;""","""e) rifiuto organico : i rifiuti biodegradabili di giardini e parchi, i rifiuti alimentari e di cucina prodotti da nuclei domestici, ristoranti, servizi di ristorazione e punti di vendita al dettaglio e i rifiuti simili prodotti dall'industria alimentare raccolti in modo differenziato;""","""[]""","""PDL""","""18PDL0001550_PD.xml"""
"""#def_6""","""#autocompostaggio""","""autocompostaggio""","""il compostaggio dei propri scarti organici, effettuato da utenze domestiche o non do

In [109]:
df.select(pl.len()) # starting len

len
u32
14729


#### Null filtering

In [82]:
df.filter(pl.all_horizontal(pl.all().is_null())) # all nulls

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str


In [83]:
df.filter(pl.col('definendum').is_null())

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str
"""#def_4""","""#""",null,"""juveniles"" means:""",""""" "" juveniles"" means: specimen…","""[]""","""EurLex""","""32010R0724.xml"""
"""#def_2""","""#""",null,""""" means "" with regard to EEE, …","""(2) "" "" "" means "" with regard …","""[]""","""EurLex""","""32011L0065.xml"""
"""#def_2""","""#""",null,"""information"" means:""","""(1) "" inside information "" mea…","""[]""","""EurLex""","""32011R1227.xml"""
"""#def_1""","""#""",null,"""transitional measures provided…",""""" "" transitional measures prov…","""[]""","""EurLex""","""32012R0284.xml"""
"""#def_1""","""#""",null,"""transitional measures provided…",""""" "" transitional measures prov…","""[]""","""EurLex""","""32012R0996.xml"""
…,…,…,…,…,…,…,…
"""#def_1""","""#""",null,"""composite product"" means compo…",""""" "" composite product"" means c…","""['/akn/eu/act/decision/2007/27…","""EurLex""","""32019R0759.xml"""
"""#def_7""","""#""",null,""""" means the place where the co…",""""" "" "" means the place where th…","""[]""","""EurLex""","""32019R1602.xml"""
"""#def_7""","""#""",null,""""" has not been reported "" mean…","""(7) "" "" "" has not been reporte…","""[]""","""EurLex""","""32020R0688.xml"""


In [84]:
df.filter(pl.col('definiens').is_null())

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str
"""#def_2""","""#portaleDeiServiziTelematici""","""portale dei servizi telematici""",null,"""b) portale dei servizi telemat…","""[]""","""Normattiva""","""20110418_011G0087_VIGENZA_2024…"
"""#def_8""","""#incaricoGliIncarichi""","""«incarico», gli incarichi""",null,"""h) «incarico», gli incarichi :""","""[]""","""Normattiva""","""20201215_20G00190_ORIGINALE.xm…"
"""#def_19""","""#enteCreditizio""","""ente creditizio""",null,"""s) «ente creditizio» : ((il so…","""[]""","""Normattiva""","""20230317_23G00035_VIGENZA_2023…"


In [85]:
df.filter(pl.col('label').is_null())

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str


In [86]:
df = df.drop_nulls()
df.select(pl.len())


len
u32
16492


#### Drop duplicates

In [87]:
df = df.unique()
df.select(pl.len())

len
u32
16490


In [91]:

pl.Config.set_tbl_rows(30)
pl.Config.set_fmt_str_lengths(300)
#df.filter(pl.col('showAs') == 'insect')
df.filter(pl.col('definendum').str.contains('insects'))

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str
"""#def_32""","""#insects""","""insects""","""means insects as defined in point (17) of Article 2 of Regulation (EU) 2019/625""","""(32) "" insects "" means insects as defined in point (17) of Article 2 of Regulation (EU) 2019/625""","""['/akn/eu/act/regulation/2019/625/!main~art_2', '/akn/eu/act/regulation/2019/625/!main']""","""EurLex""","""32021R0405.xml"""
"""#def_4""","""#insects""","""insects""","""means insects as defined in point (17) of Article 2 of Regulation (EU) 2019/625""","""(4) "" insects "" means insects as defined in point (17) of Article 2 of Regulation (EU) 2019/625""","""['/akn/eu/act/regulation/2019/625/!main~art_2', '/akn/eu/act/regulation/2019/625/!main']""","""EurLex""","""32020R2235.xml"""
"""#def_17""","""#insects""","""insects""","""means food consisting of, isolated from or produced from insects or their parts including any life stadia of insects intended for human consumption which are, when applicable, authorised in accordance with Regulation (EU) 2015/2283 and listed in Implementing Regulation (EU) 2017/2470""","""(17) "" insects "" means food consisting of, isolated from or produced from insects or their parts including any life stadia of insects intended for human consumption which are, when applicable, authorised in accordance with Regulation (EU) 2015/2283 and listed in Implementing Regulation (EU) 2017/247…","""['/akn/eu/act/regulation/2015/2283/!main', '/akn/eu/act/regulation/2017/2470/!main']""","""EurLex""","""32019R0625.xml"""


### Unwanted characters

"«", " ", "'"

In [98]:
df.filter(pl.col('definendum').str.starts_with("normativa di "))

def_n,label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str,str
"""#def_13""","""#normativaDiArmonizzazioneDellUnione""","""normativa di armonizzazione dell'Unione""","""qualunque normativa dell'Unione che armonizza le condizioni di commercializzazione dei prodotti;""","""o) «normativa di armonizzazione dell'Unione» : qualunque normativa dell'Unione che armonizza le condizioni di commercializzazione dei prodotti;""","""[]""","""Normattiva""","""20160525_16G00096_ORIGINALE.xml"""
"""#def_4""","""#normativaDiArmonizzazioneDellUnione""","""normativa di armonizzazione dell'Unione""","""normativa dell'Unione europea che armonizza le condizioni di commercializzazione del dispositivo antiabbandono quale prodotto;""","""d) normativa di armonizzazione dell'Unione : normativa dell'Unione europea che armonizza le condizioni di commercializzazione del dispositivo antiabbandono quale prodotto;""","""[]""","""Normattiva""","""20191023_19G00130_ORIGINALE.xml"""
"""#def_25""","""#normativaDiArmonizzazioneDellUnione""","""normativa di armonizzazione dell'Unione""","""la normativa dell'Unione che armonizza le condizioni di commercializzazione dei prodotti;""","""dd) «normativa di armonizzazione dell'Unione» : la normativa dell'Unione che armonizza le condizioni di commercializzazione dei prodotti;""","""[]""","""Normattiva""","""20160714_16G00137_ORIGINALE.xml"""
"""#def_23""","""#normativaDiArmonizzazioneDellUnione""","""normativa di armonizzazione dell'Unione""","""la normativa dell'Unione europea che armonizza le condizioni di commercializzazione dei prodotti;""","""bb) normativa di armonizzazione dell'Unione : la normativa dell'Unione europea che armonizza le condizioni di commercializzazione dei prodotti;""","""[]""","""Normattiva""","""20160525_16G00099_ORIGINALE.xml"""
"""#def_21""","""#normativaDiArmonizzazioneDellUnioneEuropea""","""normativa di armonizzazione dell'Unione europea""","""la normativa dell'Unione europea che armonizza le condizioni di commercializzazione dei prodotti;""","""z) «normativa di armonizzazione dell'Unione europea» : la normativa dell'Unione europea che armonizza le condizioni di commercializzazione dei prodotti;""","""[]""","""Normattiva""","""20150812_15G00141_ORIGINALE.xml"""
"""#def_30""","""#normativaDiArmonizzazioneDellUnioneEuropea""","""normativa di armonizzazione dell'Unione europea""","""la normativa dell'Unione europea che armonizza le condizioni di commercializzazione dei prodotti.""","""ii) normativa di armonizzazione dell'Unione europea : la normativa dell'Unione europea che armonizza le condizioni di commercializzazione dei prodotti.""","""[]""","""Normattiva""","""20160111_16G00001_VIGENZA_20201222.xml"""


In [100]:
df.group_by('definendum').agg(pl.len()).sort('len', descending=True)

definendum,len
str,u32
"""credit institution""",193
"""competent authority""",60
"""placing on the market""",48
"""manufacturer""",47
"""quota""",36
"""making available on the market""",36
"""importer""",34
"""distributor""",33
"""Ministero""",32


---

In [2]:
import polars as pl

df = pl.read_parquet('../data/definitions_corpus/definitions.parquet')

dictu = df.with_columns(pl.col('references').list.len().alias('references_len')).filter(pl.col('references_len') > 0).select(pl.col('references')).to_dict()
list = dictu['references'].to_list()
flat_list = [item for sublist in list for item in sublist]
df

id,definition_text,def_n,dataset,document_id,references,frbr_work,frbr_expression
u32,str,str,str,str,list[str],str,str
0,"""obstacles to trade: means any …","""#def_1""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
1,"""injury: means any material inj…","""#def_2""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
2,"""adverse trade effects: means t…","""#def_3""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
3,"""Union industry: means either: …","""#def_4""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
4,"""Union enterprise: means a comp…","""#def_5""","""EurLex""","""32015R1843.xml""",[],"""/akn/eu/act/regulation/2015-10…","""/akn/eu/act/regulation/2015-10…"
…,…,…,…,…,…,…,…
16517,"""contratto di sponsorizzazione:…","""#def_8""","""PDL""","""18PDL0040890_PD.xml""",[],"""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…"
16518,"""rappresentanti di interessi: i…","""#def_1""","""PDL""","""19PDL0004650_PD.xml""","[""#item_b""]","""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…"
16519,"""decisori pubblici: i membri de…","""#def_2""","""PDL""","""19PDL0004650_PD.xml""",[],"""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…"


In [3]:
#check work and expression max length
df.with_columns(pl.col('frbr_expression').str.len_chars().alias('fds')).sort('fds', descending=True)

id,definition_text,def_n,dataset,document_id,references,frbr_work,frbr_expression,fds
u32,str,str,str,str,list[str],str,str,u32
11603,"""addetto al recapito postale: i…","""#def_1""","""Normattiva""","""20220606_22G00067_ORIGINALE.xm…","[""/akn/it/act/decretoLegge/stato/2020-07-17/76/!main#art_26-com7"", ""/akn/it/act/legge/stato/2020-09-11/120/!main""]","""/akn/it/act/decreto/presidenza…","""/akn/it/act/decreto/presidenza…",116
11604,"""app IO: il punto di accesso te…","""#def_2""","""Normattiva""","""20220606_22G00067_ORIGINALE.xm…","[""/akn/it/act/decretoLegislativo/stato/2005-03-07/82/!main#art_64bis""]","""/akn/it/act/decreto/presidenza…","""/akn/it/act/decreto/presidenza…",116
11605,"""avviso di cortesia: l'avviso i…","""#def_3""","""Normattiva""","""20220606_22G00067_ORIGINALE.xm…","[""/akn/it/act/decretoLegge/stato/2020/76/!main#art_26-com5bis"", ""/akn/it/act/decretoLegge/stato/2020/76/!main#art_26-com6"", … ""/akn/it/act/legge/stato/2020-09-11/120/!main""]","""/akn/it/act/decreto/presidenza…","""/akn/it/act/decreto/presidenza…",116
11606,"""CAD: ildecreto legislativo 7 m…","""#def_4""","""Normattiva""","""20220606_22G00067_ORIGINALE.xm…","[""/akn/it/act/decretoLegislativo/stato/2005-03-07/82/!main""]","""/akn/it/act/decreto/presidenza…","""/akn/it/act/decreto/presidenza…",116
11607,"""CIE: il documento d'identita' …","""#def_5""","""Normattiva""","""20220606_22G00067_ORIGINALE.xm…","[""/akn/it/act/decretoLegislativo/stato/2005-03-07/82/!main#art_66""]","""/akn/it/act/decreto/presidenza…","""/akn/it/act/decreto/presidenza…",116
…,…,…,…,…,…,…,…,…
16133,"""attività di rappresentanza di …","""#def_1""","""PDL""","""18PDL0153920_PD.xml""","[""#item_b""]","""/akn/it/bill/legge/2018-03-23/…","""/akn/it/bill/legge/2018-03-23/…",44
16134,"""rappresentanti di interessi: i…","""#def_2""","""PDL""","""18PDL0153920_PD.xml""","[""#item_d""]","""/akn/it/bill/legge/2018-03-23/…","""/akn/it/bill/legge/2018-03-23/…",44
16135,"""portatori di interessi: person…","""#def_3""","""PDL""","""18PDL0153920_PD.xml""",[],"""/akn/it/bill/legge/2018-03-23/…","""/akn/it/bill/legge/2018-03-23/…",44


In [4]:
df.with_columns(pl.col('references').list.len().alias('references_len')).filter(pl.col('references_len') > 0)

id,definition_text,def_n,dataset,document_id,references,frbr_work,frbr_expression,references_len
u32,str,str,str,str,list[str],str,str,u32
7,"""authorisation: means authorisa…","""#def_2""","""EurLex""","""32019L2034.xml""","[""/akn/eu/act/directive/2014/65/!main~art_5"", ""/akn/eu/act/directive/2014/65/!main""]","""/akn/eu/act/directive/2019-11-…","""/akn/eu/act/directive/2019-11-…",2
8,"""branch: means a branch as defi…","""#def_3""","""EurLex""","""32019L2034.xml""","[""/akn/eu/act/directive/2014/65/!main~art_4(1)"", ""/akn/eu/act/directive/2014/65/!main""]","""/akn/eu/act/directive/2019-11-…","""/akn/eu/act/directive/2019-11-…",2
9,"""close links: means close links…","""#def_4""","""EurLex""","""32019L2034.xml""","[""/akn/eu/act/directive/2014/65/!main~art_4(1)"", ""/akn/eu/act/directive/2014/65/!main""]","""/akn/eu/act/directive/2019-11-…","""/akn/eu/act/directive/2019-11-…",2
11,"""commodity and emission allowan…","""#def_6""","""EurLex""","""32019L2034.xml""","[""/akn/eu/act/regulation/2013/575/!main~art_4(1)"", ""/akn/eu/act/regulation/2013/575/!main""]","""/akn/eu/act/directive/2019-11-…","""/akn/eu/act/directive/2019-11-…",2
12,"""control: means the relationshi…","""#def_7""","""EurLex""","""32019L2034.xml""","[""/akn/eu/act/directive/2013/34/!main~art_22"", ""/akn/eu/act/directive/2013/34/!main"", … ""/akn/eu/documentCollection/L/gu/2002-09-11/243/!main#eop_1""]","""/akn/eu/act/directive/2019-11-…","""/akn/eu/act/directive/2019-11-…",9
…,…,…,…,…,…,…,…,…
16493,"""gestore di piattaforme web: il…","""#def_2""","""PDL""","""18PDL0061100_PD.xml""","[""/akn/it/act/codiceCivile/stato/1942-03-16/262/!main#art_2359"", ""#item_a""]","""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…",2
16506,"""paesaggio minerario: una porzi…","""#def_6""","""PDL""","""18PDL0032690_PD.xml""","[""#art_131"", ""/akn/it/act/decretoLegislativo/stato/2004-01-22/42/!main"", ""#art_136__para_1__item_a""]","""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…",3
16509,"""«assistito» la persona assisti…","""#def_1""","""PDL""","""19PDL0003440_PD.xml""","[""/akn/it/act/legge/stato/2017-12-27/205/!main#art_1__para_255""]","""/akn/it/bill/propostaDiLegge/2…","""/akn/it/bill/propostaDiLegge/2…",1


In [8]:
df.filter(pl.col('definition_text').str.len_chars() < 20)

id,definition_text,def_n,dataset,document_id,references,frbr_work,frbr_expression
u32,str,str,str,str,list[str],str,str
138,"""off-mode: off-mode""""","""#def_11""","""EurLex""","""32010R1061.xml""",[],"""/akn/eu/act/regulation/2010-09…","""/akn/eu/act/regulation/2010-09…"
1038,"""carrier: means:""","""#def_40""","""EurLex""","""32013R0952.xml""",[],"""/akn/eu/act/regulation/2013-10…","""/akn/eu/act/regulation/2013-10…"
1128,"""established: means:""","""#def_j""","""EurLex""","""32011L0061.xml""",[],"""/akn/eu/act/directive/2011-06-…","""/akn/eu/act/directive/2011-06-…"
1129,"""EU AIF: means:""","""#def_k""","""EurLex""","""32011L0061.xml""",[],"""/akn/eu/act/directive/2011-06-…","""/akn/eu/act/directive/2011-06-…"
2318,"""off-mode: off-mode""""","""#def_14""","""EurLex""","""32012R0932.xml""",[],"""/akn/eu/act/regulation/2012-10…","""/akn/eu/act/regulation/2012-10…"
…,…,…,…,…,…,…,…
8008,"""exporter: means""","""#def_19""","""EurLex""","""32015R2446.xml""",[],"""/akn/eu/act/regulation/2015-07…","""/akn/eu/act/regulation/2015-07…"
8403,"""crisis: means:""","""#def_21""","""EurLex""","""32018R1046.xml""",[],"""/akn/eu/act/regulation/2018-07…","""/akn/eu/act/regulation/2018-07…"
8954,"""off-mode: off-mode""""","""#def_11""","""EurLex""","""32010R1015.xml""",[],"""/akn/eu/act/regulation/2010-11…","""/akn/eu/act/regulation/2010-11…"


In [59]:
df.filter(pl.col('references').list.contains('/akn/it/act/decretoLegislativo/stato/2010-03-15/66/!main'))

id,definition_text,def_n,dataset,document_id,references
u32,str,str,str,str,list[str]
13913,"""codice: ildecreto legislativo …","""#def_1""","""Normattiva""","""20231113_23G00165_ORIGINALE.xm…","[""/akn/it/act/decretoLegislativo/stato/2010-03-15/66/!main""]"


In [2]:
flat_list

['/akn/eu/act/directive/ep/2013/34/~art_22',
 '/akn/eu/act/directive/ep/2013/34/',
 '/akn/eu/act/directive/ep/2013/34/',
 '/akn/eu/act/directive/ep/2006/43/',
 '/akn/eu/act/directive/ep/1978/660/',
 '/akn/eu/documentCollection/L/gu/2013-06-29/182/!main#eop_19',
 '/akn/eu/act/regulation/ep/2002/1606/',
 '/akn/eu/act/regulation/ep/2002/1606/',
 '/akn/eu/documentCollection/L/gu/2002-09-11/243/!main#eop_1',
 '/akn/eu/act/regulation/ep/2014/600/~art_2(1)',
 '/akn/eu/act/regulation/ep/2014/600/',
 '/akn/eu/act/regulation/ep/2014/600/',
 '/akn/eu/act/regulation/ep/2012/648/',
 '/akn/eu/documentCollection/L/gu/2014-06-12/173/!main#eop_84',
 '/akn/eu/act/directive/ep/2019/878/~art_3(1)',
 '/akn/eu/act/directive/ep/2019/878/',
 '/akn/eu/act/directive/ep/2019/878/',
 '/akn/eu/act/directive/ep/2013/36/',
 '/akn/eu/documentCollection/L/gu/2019-06-07/150/!main#eop_253',
 '/akn/eu/act/directive/ep/2002/87/~art_2',
 '/akn/eu/act/directive/ep/2002/87/',
 '/akn/eu/act/directive/ep/2002/87/',
 '/akn/eu/a

In [ ]:
import logging
import xml.etree.ElementTree as ET
from pathlib import Path

from LegalDefAgent.src.settings import settings
from LegalDefAgent.src.utils import setup_logging

setup_logging()


def get_xml_mapping(xml_file: Path, dataset: str):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        namespace = settings.DB_CONFIG.NAMESPACES[dataset]
        definitions_el = root.findall('.//akn:definitions', namespace)
        if not definitions_el:
            return None
        
        frbr_work = root.find('.//akn:FRBRWork', namespace)
        frbr_expression = root.find('.//akn:FRBRExpression', namespace)
                
        return {
            'frbr_work': frbr_work.find('.//akn:FRBRthis', namespace).attrib.get('value', ''),
            'frbr_expression': frbr_expression.find('.//akn:FRBRthis', namespace).attrib.get('value', ''),
            'document': xml_file
        }
    except Exception as e:
        logging.error(f"Error processing file {xml_file}: {e}")
        return None


def main():
    works = []
    expressions = []

    config = settings.DB_CONFIG
    data_dir = Path(config.XML_DATA_DIR)

    extracted, errors, total = 0, 0, 0

    for dataset in config.DATASETS:
        mapping_list = []
        logging.info(f"Mapping XML files in dataset {dataset}...")
        target = data_dir / dataset
        for file in target.rglob('*.xml'):
            total += 1
            try:
                mapping = get_xml_mapping(file, dataset)
                if mapping:
                    extracted += 1
                    works.append(mapping['frbr_work'])
                    expressions.append(mapping['frbr_expression'])
            except Exception as e:
                logging.error(f"Error processing {file}: {e}")
    
    logging.info(f'Extracted definitions from {extracted} files. {errors} errors out of {total} files.')

    return works, expressions

works, expressions = main()

In [104]:
len(works+expressions)

NameError: name 'works' is not defined

In [102]:
get_xml_mapping('/home/leo/Desktop/dhdk/Master thesis/.project/data/datasets/EurLex/31972R1369R(01).xml', 'EurLex')

NameError: name 'get_xml_mapping' is not defined

In [39]:
len(expressions)

0

In [63]:
flat_list

['/akn/eu/act/directive/ep/2013/34/~art_22',
 '/akn/eu/act/directive/ep/2013/34/',
 '/akn/eu/act/directive/ep/2013/34/',
 '/akn/eu/act/directive/ep/2006/43/',
 '/akn/eu/act/directive/ep/1978/660/',
 '/akn/eu/documentCollection/L/gu/2013-06-29/182/!main#eop_19',
 '/akn/eu/act/regulation/ep/2002/1606/',
 '/akn/eu/act/regulation/ep/2002/1606/',
 '/akn/eu/documentCollection/L/gu/2002-09-11/243/!main#eop_1',
 '/akn/eu/act/regulation/ep/2014/600/~art_2(1)',
 '/akn/eu/act/regulation/ep/2014/600/',
 '/akn/eu/act/regulation/ep/2014/600/',
 '/akn/eu/act/regulation/ep/2012/648/',
 '/akn/eu/documentCollection/L/gu/2014-06-12/173/!main#eop_84',
 '/akn/eu/act/directive/ep/2019/878/~art_3(1)',
 '/akn/eu/act/directive/ep/2019/878/',
 '/akn/eu/act/directive/ep/2019/878/',
 '/akn/eu/act/directive/ep/2013/36/',
 '/akn/eu/documentCollection/L/gu/2019-06-07/150/!main#eop_253',
 '/akn/eu/act/directive/ep/2002/87/~art_2',
 '/akn/eu/act/directive/ep/2002/87/',
 '/akn/eu/act/directive/ep/2002/87/',
 '/akn/eu/a

In [66]:
from LegalDefAgent.src.utils import resolve_reference
import re
from rapidfuzz import fuzz, process
from difflib import SequenceMatcher

flat_list2 = [re.split(r'[~#]', ref)[0] for ref in flat_list]

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

c = 0
idx = []
for i, ref in enumerate(flat_list2):
    res = process.extractOne(ref, works+expressions, scorer=fuzz.WRatio, score_cutoff=99)
    if res:
        c += 1
        idx.append(i)
        print(ref, res)

print(c)
    
    

/akn/it/act/decretoLegislativo/stato/2010-03-15/66/!main ('/akn/it/act/decretoLegislativo/stato/2010-03-15/66/!main', 100.0, 942)
/akn/it/act/decretoDelPresidenteDellaRepubblica/stato/2011-09-14/222/!main ('/akn/it/act/decretoDelPresidenteDellaRepubblica/stato/2011-09-14/222/!main', 100.0, 1101)
/akn/it/act/decretoLegislativo/stato/2017-09-15/147/!main ('/akn/it/act/decretoLegislativo/stato/2017-09-15/147/!main', 100.0, 1033)
/akn/it/act/decretoLegislativo/stato/2014-07-04/102/!main ('/akn/it/act/decretoLegislativo/stato/2014-07-04/102/!main', 100.0, 1207)
/akn/it/act/decretoLegislativo/stato/2010-01-27/39/!main ('/akn/it/act/decretoLegislativo/stato/2010-01-27/39/!main', 100.0, 938)
/akn/it/act/decretoLegislativo/stato/2010-10-13/190/!main ('/akn/it/act/decretoLegislativo/stato/2010-10-13/190/!main', 100.0, 1133)
/akn/it/act/decretoLegislativo/stato/2010-01-25/16/!main ('/akn/it/act/decretoLegislativo/stato/2010-01-25/16/!main', 100.0, 915)
/akn/it/act/decretoLegislativo/stato/2010-01

In [68]:
idx
found_list = [flat_list[i] for i in idx]
found_list

['/akn/it/act/decretoLegislativo/stato/2010-03-15/66/!main#art_44',
 '/akn/it/act/decretoDelPresidenteDellaRepubblica/stato/2011-09-14/222/!main',
 '/akn/it/act/decretoLegislativo/stato/2017-09-15/147/!main#art_23-com2',
 '/akn/it/act/decretoLegislativo/stato/2014-07-04/102/!main#art_2-com2-letn',
 '/akn/it/act/decretoLegislativo/stato/2010-01-27/39/!main',
 '/akn/it/act/decretoLegislativo/stato/2010-10-13/190/!main',
 '/akn/it/act/decretoLegislativo/stato/2010-01-25/16/!main#art_2-com1-letl',
 '/akn/it/act/decretoLegislativo/stato/2010-01-25/16/!main#art_2-com1-letl',
 '/akn/it/act/decretoLegislativo/stato/2010-01-25/16/!main#art_15',
 '/akn/it/act/decretoLegge/stato/2021-06-14/82/!main#art_5',
 '/akn/it/act/decretoDelPresidenteDellaRepubblica/stato/2014-02-19/60/!main',
 '/akn/it/act/decretoLegislativo/stato/2010-03-26/59/!main',
 '/akn/it/act/legge/stato/2016-06-28/132/!main',
 '/akn/it/act/decretoLegislativo/stato/2010-08-13/155/!main',
 '/akn/it/act/decretoLegislativo/stato/2010-0

In [58]:
len(flat_list2)

4115

---